In [ ]:
import fxcmpy
import time
import numpy as np
import pandas as pd
import threading as thr
import re
from datetime import datetime
from playsound import playsound

In [ ]:
TOKEN='eeb1a5ee55c35ee0e7b1a4571d83470cccc0eeda'
con = fxcmpy.fxcmpy(access_token=TOKEN, server='demo')
con.is_connected()

In [ ]:
def Trailing_Stop (df , Open_Price , ID , inst , Sl ,Tp, Trail,i) :
    con.subscribe_market_data(inst)
    JPY_Pattern = re.compile(r'\w{3}/JPY')  
    JPY = JPY_Pattern.findall(inst)
    ProPip = 250
    if JPY != None :
        if df['isBuy'][i] : 
            while ID not in con.get_closed_trade_ids() :
                Current_Price = con.get_last_price(inst)[1] 
                if (Current_Price - Open_Price ) * 1000 >= ProPip  :   
                    con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=Trail/10)
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                       '%s | %s |'  %(inst,timeframe) + ' Trailing Stop Changes')
                    ProPip = ProPip + 100
                    Trail = (Trail/10) + 10
            else : 
                con.unsubscribe_market_data(inst)

                    
        else : 
            while ID not in con.get_closed_trade_ids() :
                Current_Price = con.get_last_price(inst)[0] 
                if (Open_Price - Current_Price) * 1000 >= ProPip  :   
                    con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=Trail/10)
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                       '%s | %s |'  %(inst,timeframe) + ' Trailing Stop Changes')
                    ProPip = ProPip + 100
                    Trail = (Trail/10) + 10
            else : 
                con.unsubscribe_market_data(inst)
                    
            
    else :
        if df['isBuy'][i] : 
            while ID not in con.get_closed_trade_ids() :
                Current_Price = con.get_last_price(inst)[1] 
                if (Current_Price - Open_Price ) * 100 >= ProPip  :   
                    con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=Trail/10)
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                       '%s | %s |'  %(inst,timeframe) + ' Trailing Stop Changes')
                    ProPip = ProPip + 100
                    Trail = (Trail/10) + 10
            else : 
                con.unsubscribe_market_data(inst)
                
        else : 
            while ID not in con.get_closed_trade_ids() :
                Current_Price = con.get_last_price(inst)[0] 
                if (Open_Price - Current_Price) * 100 >= ProPip  :   
                    con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=Trail/10)
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                       '%s | %s |'  %(inst,timeframe) + ' Trailing Stop Changes')
                    ProPip = ProPip + 100
                    Trail = (Trail/10) + 10
            else : 
                con.unsubscribe_market_data(inst)

In [ ]:
def BUY(inst,risk,Sl,Tp, Trail):
    
    JPY_Pattern = re.compile(r'\w{3}/JPY')    
    JPY = JPY_Pattern.findall(inst)
    
    if JPY != None :
        con.subscribe_market_data('USD/JPY')
        JPY_LastPrice = int(con.get_last_price('USD/JPY')[0])
        lot = int((risk/JPY_LastPrice*10000)/Sl)
 
        order = con.create_market_buy_order(inst, lot)
        playsound('C:\\Users\\HAMED\\miniconda3\\envs\\fxcm\\Lib\\site-packages\\order.wav')
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                '%s | %s |'  %(inst,timeframe) + ' ----->  BUY Position Opened')
        df = con.get_open_positions(kind='dataframe')
        i =  len(con.get_open_positions()) - 1
        Open_Price = df.loc[i]['open']
        ID = df['tradeId'][i]
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=-(Sl/10))
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=False, rate=Tp/10)
        TrailObj = thr.Thread(target = Trailing_Stop , args =(df , Open_Price , ID , inst , Sl ,Tp, Trail,i))
        TrailObj.start()
        
    else :
        lot = int((risk/Sl)*100)
        order = con.create_market_buy_order(inst, lot)
        playsound('C:\\Users\\HAMED\\miniconda3\\envs\\fxcm\\Lib\\site-packages\\order.wav')
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                '%s | %s |'  %(inst,timeframe) + ' ----->  BUY Position Opened')
        df = con.get_open_positions(kind='dataframe')
        i =  len(con.get_open_positions()) - 1
        Open_Price = df.loc[i]['open']
        ID = df['tradeId'][i]
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=-(Sl/10))
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=False, rate=Tp/10)
        TrailObj = thr.Thread(target = Trailing_Stop , args =(df , Open_Price , ID , inst , Sl ,Tp, Trail,i))
        TrailObj.start()     

In [ ]:
def SELL(inst,risk,Sl,Tp,Trail):
    
    JPY_Pattern = re.compile(r'\w{3}/JPY')    
    JPY = JPY_Pattern.findall(inst)
        
    if JPY != None :
        con.subscribe_market_data('USD/JPY')
        JPY_LastPrice = int(con.get_last_price('USD/JPY')[0])
        lot = int((risk/JPY_LastPrice*10000)/Sl)
 

        order = con.create_market_sell_order(inst, lot)
        playsound('C:\\Users\\HAMED\\miniconda3\\envs\\fxcm\\Lib\\site-packages\\order.wav')
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                '%s | %s |'  %(inst,timeframe) + ' ----->  BUY Position Opened')
        df = con.get_open_positions(kind='dataframe')
        i =  len(con.get_open_positions()) - 1
        Open_Price = df.loc[i]['open']
        ID = df['tradeId'][i]
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=-(Sl/10))
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=False, rate=Tp/10)
        TrailObj = thr.Thread(target = Trailing_Stop , args =(df , Open_Price , ID , inst , Sl ,Tp, Trail,i))
        TrailObj.start()
        
    else :
        lot = int((risk/Sl)*100)
        order = con.create_market_sell_order(inst, lot)
        playsound('C:\\Users\\HAMED\\miniconda3\\envs\\fxcm\\Lib\\site-packages\\order.wav')
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                '%s | %s |'  %(inst,timeframe) + ' ----->  BUY Position Opened')
        df = con.get_open_positions(kind='dataframe')
        i =  len(con.get_open_positions()) - 1
        Open_Price = df.loc[i]['open']
        ID = df['tradeId'][i]
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=-(Sl/10))
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=False, rate=Tp/10)
        TrailObj = thr.Thread(target = Trailing_Stop , args =(df , Open_Price , ID , inst , Sl ,Tp, Trail,i))
        TrailObj.start()

In [ ]:
def UpTrend(timeframe, inst, lastBID_open,risk,Sl,Tp, Trail):
    data = con.get_candles(instrument=inst, period=timeframe, number=26)
    if data['askclose'][-1] > data['askhigh'][0] :
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                '%s | %s | ChikouSpan is Up -- BUY Order is Taking...'  %(inst,timeframe))
        BUY(inst,risk,Sl,Tp, Trail)
        
    else : 
        while ICHI(timeframe,inst)[0] == 'up':    
            data = con.get_candles(instrument=inst, period=timeframe, number=26)
            if  lastBID_open != data['bidopen'][-1]: 
                if data['askclose'][-1] > data['askhigh'][0] :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                          '%s | %s | ChikouSpan is Up -- BUY Order is Taking...'  %(inst,timeframe))
                    BUY(inst,risk,Sl,Tp, Trail)
                    break
                else :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                          '%s | %s | Waiting for ChikouSpan Validation...'  %(inst,timeframe))
                    lastBID_open = data['bidopen'][-1]
            else :
                if timeframe == 'm1' :
                    time.sleep(60)
                elif timeframe == 'm5' :
                    time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
                elif timeframe == 'm15' :
                    time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
                elif timeframe == 'm30' :
                    time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
                elif timeframe == 'H1' :
                    time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)

In [ ]:
def DownTrend(timeframe, inst, lastBID_open,risk,Sl,Tp, Trail):
    data = con.get_candles(instrument=inst, period=timeframe, number=26)
    if data['bidclose'][-1] < data['bidlow'][0] :
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
              '%s | %s | ChikouSpan is Down -- SELL Order is Taking...'  %(inst,timeframe))
        SELL(inst,risk,Sl,Tp, Trail)
        
    else : 
        while ICHI(timeframe,inst)[0] == 'down':    
            data = con.get_candles(instrument=inst, period=timeframe, number=26)
            if  lastBID_open != data['bidopen'][-1]: 
                if data['bidclose'][-1] < data['bidlow'][0] :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                          '%s | %s | ChikouSpan is Down -- SELL Order is Taking...'  %(inst,timeframe))
                    SELL(inst,risk,Sl,Tp, Trail)
                    break
                else :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                          '%s | %s | Waiting for ChikouSpan Validation...'  %(inst,timeframe))
                    lastBID_open = data['bidopen'][-1]
            else :
                if timeframe == 'm1' :
                    time.sleep(60)
                elif timeframe == 'm5' :
                    time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
                elif timeframe == 'm15' :
                    time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
                elif timeframe == 'm30' :
                    time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
                elif timeframe == 'H1' :
                    time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)

In [ ]:
def Road(timeframe, inst,lastBID_open, risk, Sl,Tp, Kijunsen,crossNum, Trail): 
    number = 27    
    while True :
        data = con.get_candles(instrument=inst, period=timeframe, number=number)
        HstHigh_tk = np.array(data)[17:26,2].max()   
        LstLow_tk = np.array(data)[17:26,3].min()             
        HstHigh_kj = np.array(data)[:26,2].max()      
        LstLow_kj = np.array(data)[:26,3].min()
        Tenkensen_Old = (HstHigh_tk + LstLow_tk)/2 
        Kijunsen_Old = (HstHigh_kj + LstLow_kj)/2            
        if Tenkensen_Old < Kijunsen_Old : 
            print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                    '%s | %s | UpRoad(%s) -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum))
            UpRoad= thr.Thread(target= Bull_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum, Trail))
            UpRoad.start()
            break
                
        elif Tenkensen_Old > Kijunsen_Old : 
            print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                    '%s | %s | DownRoad(%s) -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum))
            DownRoad= thr.Thread(target= Bear_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum, Trail))
            DownRoad.start()
            break
                
        else :
            number = number + 1 

In [ ]:
def Bear_Hunter (timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum, Trail):
    while True :
        if ICHI(timeframe,inst)[0] == 'down' or ICHI(timeframe,inst)[0] == 'road' :
            data = con.get_candles(instrument=inst, period=timeframe, number=26)
            if  lastBID_open != data['bidopen'][-1] :  
                HstHigh_kj = np.array(data)[:26,2].max()      
                LstLow_kj = np.array(data)[:26,3].min()
                KijunSen_new = (HstHigh_kj + LstLow_kj)/2       
                if KijunSen_new < Kijunsen:
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
                          '%s | %s | Cross(%s) -- Kjijunsen Shifted Down'  %(inst,timeframe,crossNum))
                    Chikou_Down_Thread = thr.Thread(target=DownTrend, args= (timeframe, inst, lastBID_open,risk,Sl,Tp, Trail))
                    Chikou_Down_Thread.start()                                 
                    break
                   
                else:
                    lastBID_open = data['bidopen'][-1]
                    
            else :
                if timeframe == 'm1' :
                    time.sleep(60)
                elif timeframe == 'm5' :
                    time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
                elif timeframe == 'm15' :
                    time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
                elif timeframe == 'm30' :
                    time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
                elif timeframe == 'H1' :
                    time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)   
                
        else :
            print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
              '%s | %s | Cross(%s) Failed -- Waiting for Another Cross...'  %(inst,timeframe,crossNum)) 
            break    

In [ ]:
def Bull_Hunter (timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum, Trail):
    while True :
        if ICHI(timeframe,inst)[0] == 'up' or ICHI(timeframe,inst)[0] == 'road' :
            data = con.get_candles(instrument=inst, period=timeframe, number=26)
            if  lastBID_open != data['bidopen'][-1] :       
                HstHigh_kj = np.array(data)[:26,2].max()      
                LstLow_kj = np.array(data)[:26,3].min()
                KijunSen_new = (HstHigh_kj + LstLow_kj)/2            
                if KijunSen_new > Kijunsen:
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
                      '%s | %s | Cross(%s) -- Kjijunsen Shifted Up'  %(inst,timeframe,crossNum))
                    Chikou_Up_Thread = thr.Thread(target=UpTrend, args= (timeframe, inst, lastBID_open,risk,Sl,Tp, Trail))
                    Chikou_Up_Thread.start()
                    break
                                
                else:
                    lastBID_open = data['bidopen'][-1]
                    
            else :
                if timeframe == 'm1' :
                    time.sleep(60)
                elif timeframe == 'm5' :
                    time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
                elif timeframe == 'm15' :
                    time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
                elif timeframe == 'm30' :
                    time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
                elif timeframe == 'H1' :
                    time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)
    
        else :
            print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
              '%s | %s | Cross(%s) Failed -- Waiting for Another Cross...'  %(inst,timeframe,crossNum)) 
            break      

In [ ]:
def ICHI(timeframe,inst) :
    data = con.get_candles(instrument=inst, period=timeframe, number=26)
    HstHigh_tk = np.array(data)[17:,2].max()   
    LstLow_tk = np.array(data)[17:,3].min()             
    HstHigh_kj = np.array(data)[:26,2].max()      
    LstLow_kj = np.array(data)[:26,3].min()
    TenkenSen = (HstHigh_tk + LstLow_tk)/2 
    KijunSen = (HstHigh_kj + LstLow_kj)/2
         
    if KijunSen < TenkenSen:
        ichi = 'up'
        return ichi , KijunSen , TenkenSen                             
                        
    elif KijunSen > TenkenSen:
        ichi = 'down'
        return ichi , KijunSen , TenkenSen                           
       
    else:
        ichi = 'road'
        return ichi , KijunSen , TenkenSen    

In [ ]:
def Ichimoku(timeframe,inst,lastBID_open):
    while True:
        data = con.get_candles(instrument=inst, period=timeframe, number=26) 
        if  lastBID_open != data['bidopen'][-1] :
            HstHigh_tk = np.array(data)[17:,2].max()   
            LstLow_tk = np.array(data)[17:,3].min()           
            HstHigh_kj = np.array(data)[:26,2].max()      
            LstLow_kj = np.array(data)[:26,3].min()
            KijunSen = (HstHigh_kj + LstLow_kj)/2
            TenkenSen = (HstHigh_tk + LstLow_tk)/2
            
            if KijunSen < TenkenSen:
                ichi = 'up'
                return ichi , KijunSen , TenkenSen                                       
            
            elif KijunSen > TenkenSen:
                ichi = 'down'
                return ichi , KijunSen , TenkenSen                             
    
            else:
                ichi = 'road'
                return ichi , KijunSen , TenkenSen   
            
        else :
            
            if timeframe == 'm1':
                time.sleep(60)
            elif timeframe == 'm5' :
                time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
            elif timeframe == 'm15' :
                time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
            elif timeframe == 'm30' :
                time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
            elif timeframe == 'h1' :
                time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)

In [ ]:
def SwitchFounder(timeframe,inst,risk,Sl,Tp, Trail):
    road = []
    crossNum = 0
    while True:          
        lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
        Trend = Ichimoku(timeframe,inst,lastBID_open)
        if Trend[0] == 'down':
            
            while True:
                lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                Trend = Ichimoku(timeframe,inst,lastBID_open)
                if Trend[0] == 'up':
                    lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                    crossNum = crossNum + 1
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                          '%s | %s | Cross(%s) Found -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum)) 
                    Kijunsen = Trend[1]
                    road.clear()
                    UpObj= thr.Thread(target= Bull_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum, Trail))
                    UpObj.start()
                    break  
                    
                elif Trend[0] == 'road' : 
                    lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                    crossNum = crossNum + 1
                    print ('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
                             '%s | %s | RoadCross(%s) Found -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum))
                    Kijunsen = Trend[1]
                    road.append(inst)
                    UpObj= thr.Thread(target= Bull_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum, Trail))
                    UpObj.start()
                    break
   
                    
                else :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + '%s | %s | NOT MATCH | ' %(inst,timeframe) + 
                          '%.2f' %(100 -((Trend[1]-Trend[2])*250)) + ' D')                            
            
        elif Trend[0] == 'up':
            
            while True:
                lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                Trend = Ichimoku(timeframe,inst,lastBID_open)
                if Trend[0] == 'down':
                    lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                    crossNum = crossNum + 1
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                          '%s | %s | Cross(%s) Found -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum)) 
                    Kijunsen = Trend[1]
                    road.clear()
                    DownObj= thr.Thread(target= Bear_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum, Trail))
                    DownObj.start()
                    break 
                    
                elif Trend[0] == 'road' : 
                    lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                    crossNum = crossNum + 1
                    print ('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
                             '%s | %s | RoadCross(%s) Found -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum))
                    Kijunsen = Trend[1]
                    road.append(inst)
                    DownObj= thr.Thread(target= Bear_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum, Trail))
                    DownObj.start()
                    break
        
                else :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + '%s | %s | NOT MATCH | ' %(inst,timeframe) + 
                          '%.2f' %(100 -((Trend[2]-Trend[1])*250)) + ' U')
                                      
        else :
            if inst not in road :   
                road.append(inst) 
                crossNum = crossNum + 1
                lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                Kijunsen = Trend[1]
                RoadObj = thr.Thread(target= Road, args= (timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum, Trail))
                RoadObj.start()

In [ ]:
currency = ['GBP/JPY' , 'USD/JPY' , 'AUD/USD' , 'GBP/USD',
            'USD/CAD' , 'NZD/USD', 'EUR/USD', 'USD/CHF']
risk = 20  # USD
Sl = 200
Tp = 500
Trail = 100
timeframe = 'm5'
print('Connecting to Server...\n')
while True :
    extime = int(datetime.now().strftime("%M"))
    if extime%int(timeframe[1:]) == 0 :
        for i in range(int(len(currency))) :
            inst = currency[i]
            tradeObj = thr.Thread(target = SwitchFounder , args=(timeframe,inst,risk,Sl,Tp, Trail))
            tradeObj.start()  
        print('Start Hunting...\n')
        break
    else :
        time.sleep(30)

df = con.get_open_positions(kind='dataframe')
for i in range(len(con.get_open_positions())) :
    Open_Price = df.loc[i]['open']
    ID = df['tradeId'][i]
    inst = df.loc[i]['currency']            
    TrailObj = thr.Thread(target = Trailing_Stop , args =(df , Open_Price , ID , inst , Sl ,Tp, Trail,i))
    TrailObj.start()
    
print('There are %s Opened Positions. \n' %(len(con.get_open_positions())))
    